In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from datasets import Dataset
import polars as pl
import json



Mounted at /content/drive


In [2]:
#from datasets import load_dataset, Dataset

sheet_url="/content/drive/MyDrive/Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv"


In [ ]:
'''
Analytical Plan



1. Exploratory Data Analysis (EDA)
Before modeling, perform in-depth EDA to understand:


Data Distribution: How many unique intents, categories, and flags exist? Are they balanced?

Text Analysis:
Length of instruction and response (word count, character count).
Most frequent words/n-grams per intent (using word clouds or bar plots).

Flag Analysis:
TBA

2. Intent Prediction Models (Easy → Complex)
Baseline Models (Easy)

'''

In [4]:
df = pl.read_csv(
    sheet_url,
)

In [5]:
respon = df

In [59]:
print(respon.schema)
2
respon.glimpse()

Schema({'flags': String, 'instruction': String, 'category': String, 'intent': String, 'response': String, 'instruction_char_count': UInt32, 'instruction_word_count': UInt32, 'response_char_count': UInt32, 'response_word_count': UInt32})
Rows: 26872
Columns: 9
$ flags                  <str> 'B', 'BQZ', 'BLQZ', 'BL', 'BCELN', 'BI', 'BCLN', 'BL', 'BL', 'BLQ'
$ instruction            <str> 'question about cancelling order {{Order Number}}', 'i have a question about cancelling oorder {{Order Number}}', 'i need help cancelling puchase {{Order Number}}', 'I need to cancel purchase {{Order Number}}', 'I cannot afford this order, cancel purchase {{Order Number}}', 'can you help me cancel order {{Order Number}}?', 'I can no longer afford order {{Order Number}}, cancel it', 'I am trying to cancel purchase {{Order Number}}', 'I have got to cancel purchase {{Order Number}}', 'i need help canceling purchase {{Order Number}}'
$ category               <str> 'ORDER', 'ORDER', 'ORDER', 'ORDER', 'ORDER',

In [14]:
respon.describe()

statistic,flags,instruction,category,intent,response,instruction_char_count,instruction_word_count,response_char_count,response_word_count
str,str,str,str,str,str,f64,f64,f64,f64
"""count""","""26872""","""26872""","""26872""","""26872""","""26872""",26872.0,26872.0,26872.0,26872.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,46.889513,8.711484,634.104495,103.138806
"""std""",null,null,null,null,null,10.897578,2.605192,331.593822,50.735034
"""min""","""B""","""ETA of oorder {{Order Number}}""","""ACCOUNT""","""cancel_order""","""""I completely understand the i…",6.0,1.0,57.0,9.0
"""25%""",null,null,null,null,null,40.0,7.0,427.0,72.0
"""50%""",null,null,null,null,null,48.0,9.0,540.0,90.0
"""75%""",null,null,null,null,null,55.0,11.0,753.0,121.0
"""max""","""BZ""","""you arenot being helpful at al…","""SUBSCRIPTION""","""track_refund""","""{{Delivery City}}, a city stee…",92.0,16.0,2472.0,394.0


In [15]:
respon.group_by("intent").agg([
  pl.col("instruction").count().alias("count_of_intent")
])

intent,count_of_intent
str,u32
"""change_shipping_address""",973
"""edit_account""",1000
"""track_order""",995
"""delivery_options""",995
"""get_refund""",997
…,…
"""change_order""",997
"""newsletter_subscription""",999
"""registration_problems""",999


In [104]:
def analyze_data_distribution(df):
  results = {
      'shape': respon.shape,
      'total_samples': respon.height,
      'intents': {
      'unique_count': respon['intent'].n_unique(),
      'value_counts': respon['intent'].value_counts(sort=True),
      'is_balanced': None
      },
      'categories': {
      'unique_count': respon['category'].n_unique(),
      'value_counts': respon['category'].value_counts(sort=True),
      'is_balanced': None
      },
      'flags': {
      'unique_count': respon['flags'].n_unique(),
      'value_counts': respon['flags'].value_counts(sort=True),
      'is_balanced': None
      }
  }


  list(results.keys())
  # Determine if distributions are balanced (heuristic: no class > 50% of total)
  for col in ['intents', 'categories', 'flags']:
    value_counts = results[col]['value_counts']
    #print(value_counts)
    max_percentage = (value_counts['count'].max() / results['total_samples']) * 100
    #print(f"max_percentage:")
    #print(max_percentage)
    results[col]['is_balanced'] = max_percentage < 50


  return results



def print_analysis_results(results):
  print(f"Total samples: {results['total_samples']}\n")

  for col in ['intents', 'categories', 'flags']:
    print(f"=== {col.upper()} ===")
    print(f"Unique {col}: {results[col]['unique_count']}")
    print(f"Distribution (top 5):")
    print(results[col]['value_counts'].head(5))
    print(f"Is balanced: {'Yes' if results[col]['is_balanced'] else 'No'}")
    print("\n")


In [107]:
results=analyze_data_distribution(respon)

print_analysis_results(results)

Total samples: 26872

=== INTENTS ===
Unique intents: 27
Distribution (top 5):
shape: (5, 2)
┌──────────────────────────┬───────┐
│ intent                   ┆ count │
│ ---                      ┆ ---   │
│ str                      ┆ u32   │
╞══════════════════════════╪═══════╡
│ check_invoice            ┆ 1000  │
│ complaint                ┆ 1000  │
│ contact_customer_service ┆ 1000  │
│ edit_account             ┆ 1000  │
│ switch_account           ┆ 1000  │
└──────────────────────────┴───────┘
Is balanced: Yes


=== CATEGORIES ===
Unique categories: 11
Distribution (top 5):
shape: (5, 2)
┌──────────┬───────┐
│ category ┆ count │
│ ---      ┆ ---   │
│ str      ┆ u32   │
╞══════════╪═══════╡
│ ACCOUNT  ┆ 5986  │
│ ORDER    ┆ 3988  │
│ REFUND   ┆ 2992  │
│ INVOICE  ┆ 1999  │
│ CONTACT  ┆ 1999  │
└──────────┴───────┘
Is balanced: Yes


=== FLAGS ===
Unique flags: 394
Distribution (top 5):
shape: (5, 2)
┌───────┬───────┐
│ flags ┆ count │
│ ---   ┆ ---   │
│ str   ┆ u32   │
╞═══════╪═════

| Component              | First Block                     | Second Block                                           |
| ---------------------- | ------------------------------- | ------------------------------------------------------ |
| Data analyzed          | `instruction`                   | `response`                                             |
| Output dictionary name | `instruction_ngrams_per_intent` | `response_ngrams_per_intent`                           |
| Print statement labels | "Top Unigrams / Bigrams"        | "Top Unigrams in Responses / Top Bigrams in Responses" |


**N-grams & Text lengths**:
#####Calculate text lengths for `instruction` and `response` columns (word count and character count), then for each unique `intent`, analyze the most frequent words and n-grams in both 'instruction' and 'response' texts.






In [115]:
##### Add new columns to the DataFrame for word count and character count for both `instruction` and `response` columns.
##### Use Polars string methods to count characters and words, and then add these as new columns to the `respon` DataFrame.

respon = respon.with_columns(
    [pl.col('instruction').str.len_chars().alias('instruction_char_count'),
     pl.col('instruction').str.split(' ').list.len().alias('instruction_word_count'),
     pl.col('response').str.len_chars().alias('response_char_count'),
     pl.col('response').str.split(' ').list.len().alias('response_word_count')]
)


with pl.Config(fmt_str_lengths=10000, tbl_width_chars=-1, tbl_cols=-1): print(respon.head(5))

shape: (5, 9)
┌───────┬──────────────────────────────────────────────────────────────┬──────────┬──────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────┬────────────────────────┬─────────────────────┬─────────────────────┐
│ flags ┆ instruction                                                  ┆ category ┆ intent       ┆ response                                                                                                                                                                                                                                                                                                                           ┆ instruction_char_count ┆ instruction_word_co

In [151]:
s=pl.Series(["I need to cancel purchase {{Order Number}}"])
print(len("I need to cancel purchase {{Order Number}}"))
s.str.split(' ').list.len()

42


""
u32
7


**Analyze frequent words/n-grams in instructions per intent**

For each unique intent, extract the 'instruction' texts and identify the most frequent words and n-grams (e.g., unigrams, bigrams) after basic text preprocessing.


In [154]:
import nltk
import re
from collections import Counter
import string
from bs4 import BeautifulSoup

# Download NLTK data
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Download 'punkt_tab'
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize






# Basic preprocessing(some incorporated from def review_to_words of previous task) and get_top_ngrams function
def get_top_ngrams(text, n=1, num_top_ngrams=10):

    text=BeautifulSoup(text, "html5lib").get_text()
    # Convert to lowercase
    text = text.lower()

    '''Puncutation handling'''
    ## Remove punctuation and numeric (if want to replace everything that isn’t a letter with spaces for tokenization.)
    #text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    #text = re.sub(r"[^a-zA-Z]", " ", text)

    # Remove punctuation (Use translate when you want to remove ASCII punctuation fast and keep digits/whitespace.)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize text
    tokens = word_tokenize(text)

    '''Stopwords handling'''
    # Remove stopwords
    '''
    - Keeps any token not in stop_words
    - No check for punctuation, digits, or token shape.
    e.g. keeps punctuation-bearing tokens,may keep numbers like 123
    '''
    #text = [w for w in text if w not in stop_words]

    # Remove stopwords
    ''''
    - Keeps only tokens made of letters/digits ( isalnum() ), and not in stop_words .
    - Drops tokens containing punctuation or symbols (e.g., "can't", "co-operate", "@user").
    e.g. may keep numbers like 123 isalnum=True, can use isalpha() if dont want number,
    isalnum() is Unicode-aware, non-ASCII letters like é count as letters.
    '''
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]



    # Stemming
    #review = [PorterStemmer().stem(w) for w in review if w not in STOPWORDS_sentiment and w not in ([n.lower() for n in namelist])]

    '''
    - Stemming text normalization technique that reduces words to a root form (a “stem”) by stripping suffixes/prefixes.
    - Stems are heuristic, not necessarily valid dictionary words (e.g., “studies” → “studi”, “connections” → “connect”).

    Might use it because of

    - Reduces vocabulary size for bag‑of‑words/TF‑IDF models.
    - Increases recall in search/IR by conflating inflected forms.
    - Faster/simpler than lemmatization; no POS tagging required.
    Trade‑offs

    - Over‑/under‑stemming can merge unrelated words or miss relations (e.g., “university” → “univers”).
    - Less linguistically accurate than lemmatization; stems can be awkward.
    - Language‑specific rules; performance varies across languages and morphological richness.
    Common Algorithms

    - Porter Stemmer: classic, conservative.
    - Snowball Stemmer: improved Porter; supports multiple languages.
    - Lancaster Stemmer: more aggressive.
    Stemming vs Lemmatization

    - Stemming: rule-based chopping; quick; approximate; output may not be a real word.
    - Lemmatization: uses vocabulary and POS (e.g., “better” → “good”); more accurate; slower.
    Examples

    - “connect”, “connected”, “connection” → “connect” (Porter/Snowball often conflate these)
    - “studies” → “studi”; “studying” → “study”
    - Lemmatization contrast: “studies” → “study”; “better” → “good”
    '''

    # Generate n-grams
    ngrams = []
    if n == 1:
        ngrams = filtered_tokens
    else:
        for i in range(len(filtered_tokens) - n + 1):
            ngrams.append(tuple(filtered_tokens[i:i+n]))

    # Count n-gram frequency
    ngram_counts = Counter(ngrams)
    return ngram_counts.most_common(num_top_ngrams)

# Get unique intents
unique_intents = respon['intent'].unique().to_list()

# Create an empty dictionary to store results
instruction_ngrams_per_intent = {}

# Loop through each unique intent
for intent in unique_intents:
    # Filter instructions for the current intent
    intent_instructions = respon.filter(pl.col('intent') == intent)['instruction'].to_list()
    # Concatenate all instruction texts
    concatenated_instructions = ' '.join(intent_instructions)

    # Get top unigrams and bigrams
    top_unigrams = get_top_ngrams(concatenated_instructions, n=1, num_top_ngrams=10)
    top_bigrams = get_top_ngrams(concatenated_instructions, n=2, num_top_ngrams=10)

    # Store results
    instruction_ngrams_per_intent[intent] = {
        'unigrams': top_unigrams,
        'bigrams': top_bigrams
    }

# Print or inspect the dictionary
for intent, data in instruction_ngrams_per_intent.items():
    print(f"\nIntent: {intent}")
    print(f"  Top Unigrams: {data['unigrams']}")
    print(f"  Top Bigrams: {data['bigrams']}")


Intent: check_payment_methods
  Top Unigrams: [('payment', 987), ('options', 340), ('methods', 329), ('help', 312), ('see', 289), ('modalities', 285), ('accepted', 246), ('allowed', 228), ('check', 227), ('available', 211)]
  Top Bigrams: [(('payment', 'options'), 330), (('payment', 'methods'), 324), (('payment', 'modalities'), 271), (('accepted', 'payment'), 201), (('allowed', 'payment'), 184), (('available', 'payment'), 175), (('see', 'payment'), 106), (('check', 'payment'), 91), (('help', 'see'), 72), (('want', 'help'), 67)]

Intent: set_up_shipping_address
  Top Unigrams: [('address', 968), ('shipping', 488), ('delivery', 476), ('set', 275), ('new', 255), ('secondary', 238), ('help', 202), ('setting', 186), ('enter', 157), ('submit', 152)]
  Top Bigrams: [(('shipping', 'address'), 478), (('delivery', 'address'), 458), (('new', 'shipping'), 124), (('new', 'delivery'), 122), (('secondary', 'shipping'), 121), (('address', 'need'), 116), (('secondary', 'delivery'), 114), (('address', 